# Kinematic Replay

In this notebook, we will demonstrate how we can replay experimentally recorded fly kinematics to infer dynamical forces.

To run this notebook on Google Colab:
- Locate the menu bar at the top of this page, go to "Runtime" → "Change runtime type", and select a GPU (e.g. "T4 GPU"). Save your setting.
- Run the first code blocks.
- Then, you can import FlyGym and start writing your own code.

If you are not using Google Colab, the first code block won't do anything when executed.

In [ ]:
try:
    import google.colab

    IN_COLAB = True
except ImportError:
    IN_COLAB = False

try:
    import flygym

    FLYGYM_INSTALLED = True
except ImportError:
    FLYGYM_INSTALLED = False

if not FLYGYM_INSTALLED:
    if IN_COLAB:
        print(
            "I'm on Colab and FlyGym is not installed. I will try to install it now. "
            "This will take a minute."
        )
        import subprocess

        subprocess.run('pip install "flygym[examples]"', shell=True)
    else:
        print(
            "FlyGym is not installed, and I'm on your own computer. I can try to "
            "install it here, but I don't want to modify your Python environment "
            "unintentionally. Please install FlyGym yourself following instructions "
            "from https://neuromechfly.org/installation.html"
        )

if IN_COLAB:
    print("In Google Colab. I will now perform some Colab-specific setups.")
    # Set up GPU a few more and rendering parameters. This should take ~1 second.
    from google.colab import files
    import distutils.util
    import os
    import subprocess

    if subprocess.run("nvidia-smi").returncode:
        raise RuntimeError(
            "Cannot communicate with GPU. "
            "Make sure you are using a GPU Colab runtime. "
            "Go to the Runtime menu and select Choose runtime type."
        )

    # Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
    # This is usually installed as part of an Nvidia driver package, but the Colab
    # kernel doesn't install its driver via APT, and as a result the ICD is missing.
    # (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
    NVIDIA_ICD_CONFIG_PATH = "/usr/share/glvnd/egl_vendor.d/10_nvidia.json"
    if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
        with open(NVIDIA_ICD_CONFIG_PATH, "w") as f:
            f.write(
                """{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
"""
            )

    # Configure MuJoCo to use the EGL rendering backend (requires GPU)
    print("Setting environment variable to use GPU rendering:")
    %env MUJOCO_GL=egl

    try:
        print("Checking that the installation succeeded:")
        import mujoco

        mujoco.MjModel.from_xml_string("<mujoco/>")
    except Exception as e:
        raise e from RuntimeError(
            "Something went wrong during installation. Check the shell output above "
            "for more information.\n"
            "If using a hosted Colab runtime, make sure you enable GPU acceleration "
            'by going to the Runtime menu and selecting "Choose runtime type".'
        )

    print("Installation successful.")

Let's check if we have the necessary data and download it if we don't.

In [ ]:
from pathlib import Path
from hashlib import md5
import urllib.request

leg_joint_angles_path = Path("./data/inverse_kinematics/leg_joint_angles.pkl")
leg_joint_angles_expected_md5 = "420d8380b2fcb9ca310f7936a11effd4"
leg_joint_angles_url = "https://github.com/NeLy-EPFL/neuromechfly-workshop/raw/refs/heads/main/data/inverse_kinematics/leg_joint_angles.pkl"


# If file doesn't exist, we need to download it
download_needed = not leg_joint_angles_path.is_file()

# If file does exist, check if it's corrupted. If so, re-download it
if not download_needed:
    with open("./data/inverse_kinematics/leg_joint_angles.pkl", "rb") as f:
        data = f.read()
        checksum = md5(data).hexdigest()
        if checksum != leg_joint_angles_expected_md5:
            download_needed = True

if download_needed:
    print("Downloading inverse kinematics data...")
    leg_joint_angles_path.parent.mkdir(exist_ok=True, parents=True)
    urllib.request.urlretrieve(leg_joint_angles_url, leg_joint_angles_path)

# Kinematic replay of untethered locomotion

We now move on to an example where we perform kinematic replay of experimentally recorded walking behaviors. Specifically, we recorded an untethered fly walking in a narrow corridor. We used the GUI of SLEAP to manually annotate ~50 frames of behavior recorded at 360 fps and downsampled to 130fps for annotations. When then leveraged the simple geometrical constraints of our setup to triangulate the 2d poses to a 3d poses. Used df3d post processing for alignment and interpolation and finally used seqikpy to perform inverse kinematics (e.g calculate the angle at each DoF). All the notebooks used to go from a raw video to joint angles can be found in this repo in the folder: 2d-3d

We will use a PD controller to actuate the DoFs of the simulated fly using these exact angles to see if the fly can walk untethered on flat terrain, as shown in the original NeuroMechFly paper (Lobato-Rios et al., *Nature Methods* 2022).

Here is the above mentioned locomotor behavior we will replay.

<video controls>
  <source src="https://github.com/NeLy-EPFL/neuromechfly-workshop/raw/refs/heads/main/data/video_data/straight_walking_bout.mp4" type="video/mp4">
</video>

Like in the previous notebook, we will perform the necessary import define some simulation parameters, format the data and visualize them.

# Data block preprocessing

## Imports

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange

from flygym import Fly, Camera, SingleFlySimulation
from flygym.preprogrammed import all_leg_dofs, all_tarsi_links

## Simulation parameters

In [ ]:
timestep = 1e-4
actuated_joints = all_leg_dofs

output_dir = Path("outputs/kinematic_replay/")
output_dir.mkdir(exist_ok=True, parents=True)

Check the format our data:

In [ ]:
def format_seqikpy_data(
    data,
    corresp_dict={"ThC": "Coxa", "CTr": "Femur", "FTi": "Tibia", "TiTa": "Tarsus1"},
):
    data_gym = {}
    for joint, values in data.items():
        if joint == "meta" or joint == "swing_stance_time":
            data_gym[joint] = values
        else:
            leg = joint[6:8]
            joint_name = joint[9:]
            seg, dof = joint_name.split("_")
            if dof == "pitch":
                newjoint = f"joint_{leg}{corresp_dict[seg]}"
            else:
                newjoint = f"joint_{leg}{corresp_dict[seg]}_{dof}"

            data_gym[newjoint] = values

    return data_gym


seq_ikpy_data_path = Path(leg_joint_angles_path)
with open(seq_ikpy_data_path, "rb") as f:
    seq_ikdata = pickle.load(f)

data = format_seqikpy_data(seq_ikdata)

data["meta"].keys()

In [ ]:
run_time = len(data["joint_RFCoxa_yaw"]) * data["meta"]["timestep"]

target_num_steps = int(run_time / timestep)
data_block = np.zeros((len(actuated_joints), target_num_steps))
input_t = np.arange(len(data["joint_LFCoxa"])) * data["meta"]["timestep"]
output_t = np.arange(target_num_steps) * timestep
for i, joint in enumerate(actuated_joints):
    data_block[i, :] = np.interp(output_t, input_t, data[joint])

print(
    "Neuromechfly has {} actuated joints and the data contains {} interpolated steps adding up to a toal of {} seconds".format(
        *data_block.shape, run_time
    )
)

In [ ]:
# The fly should walk more on the tippy toes
tarsus_offset = np.zeros(len(actuated_joints))
for i, joint in enumerate(actuated_joints):
    if "Tarsus" in joint:
        data_block[i, :] = -1 * np.pi / 5

## Visualize

In [ ]:
fig, axs = plt.subplots(
    3, 2, figsize=(8, 6), sharex=True, sharey=True, tight_layout=True
)
legs = [
    f"{side} {pos} leg"
    for pos in ["front", "middle", "hind"]
    for side in ["Left", "Right"]
]
for i, leg in enumerate(legs):
    ax = axs.flatten()[i]
    leg_code = f"{leg.split()[0][0]}{leg.split()[1][0]}".upper()
    for j, dof in enumerate(actuated_joints):
        if dof.split("_")[1][:2] != leg_code:
            continue
        ax.plot(output_t, np.rad2deg(data_block[j, :]), label=dof[8:])
    ax.set_ylim(-180, 180)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Angle (degree)")
    ax.set_yticks([-180, -90, 0, 90, 180])
    ax.set_title(leg)
    if leg == "Right front leg":
        ax.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)

fig.savefig(output_dir / "single_step.png")

In each leg 7 degrees of freedom are actuated adding up to 6x7 actuated joints.

# Run kinematic replay

Lets focus on the heart of this notebook: kinematic replay

In [ ]:
# Monitor the tarsus passive joints
tarsal_joints = [
    "_".join(["joint", tarsus_seg])
    for tarsus_seg in all_tarsi_links
    if not "Tarsus1" in tarsus_seg
]
monitored_joints = actuated_joints + tarsal_joints

# more info about those simulation parameters at neuromechfly.org
fly = Fly(
    init_pose="stretch",
    actuated_joints=actuated_joints,
    control="position",
    monitored_joints=monitored_joints,
    enable_vision=True,
    render_raw_vision=True,
)
play_speed = 0.05
# The camera parameters allows to visualize the magnitude and direction of the contact forces applied by the ground on the fly legs
cam = Camera(
    fly=fly,
    camera_id="Animat/camera_left",
    play_speed=play_speed,
    draw_contacts=True,
    play_speed_text=True,
)
sim = SingleFlySimulation(
    fly=fly,
    cameras=[cam],
)
obs, info = sim.reset()

obs_list = []
raw_vision_list = []
vision_list = []

for i in trange(target_num_steps):
    # here, we simply use the recorded joint angles as the target joint angles
    obs, reward, terminated, truncated, info = sim.step({"joints": data_block[:, i]})

    # reduce RAM usage by removing the vision data and storing it separately when rendered
    vision = obs.pop("vision")
    obs_list.append(obs.copy())
    if fly.render_raw_vision and fly._vision_update_mask[-1]:
        raw_vision_list.append(info["raw_vision"])
        vision_list.append(vision)

    sim.render()

In [ ]:
from base64 import b64encode
from IPython.display import HTML

video_name = "kinematic_replay_{}.mp4".format(cam.camera_id.split("/")[1])
if cam.draw_contacts:
    video_name = video_name.replace(".mp4", "_contacts.mp4")

output_path = output_dir / video_name
cam.save_video(output_path, stabilization_time=0)


mp4 = open(output_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(
    """
<video controls>
      <source src="%s" type="video/mp4">
</video>
"""
    % data_url
)

The first output of kinematic replay is a video that can be rendered from any different angles. Arrows at the leg tips represent the contact forces with the floor with blue being forces along the z axis, green arrows represent forces along the y axis and red arrows along the x axis.

## Observation space

In [ ]:
print(f"Observation list made {len(obs_list)} observations")
print("One observation contains information about:")
for key in obs_list[0].keys():
    try:
        print(key, obs[key].shape)
    except AttributeError:
        print(key)

Each observation contains the following information:

Joints: Data structured as a (3, 66) array. For the 66 monitored joints, it monitors the angle, angular velocity and acceleration of joint.
Fly: A (4, 3) array that describes the fly's position, velocity, rotation, and rotational velocity.
Contact Forces: A (30, 3) array that captures the 3D contact forces at each of the fly's 30 contact points (e.g 6 legs with 5 tarsal segments per leg).
End Effectors: A (6, 3) array representing the position of the fly's six end effectors.
Fly Orientation: A (3,) array containing a vector monitoring the long axis of the Thorax (e.g antero-posterior axis of the fly)
Cardinal Vectors: A (3, 3) array that contains the cardinal vectors aligned with the anterio-posterior, left-right and dorso-ventral axis.


# Exploring NeuromechFly's Observation Space and it's links to *Drosophila melanogasters* sensory capabilities

In this notebook, we e£xplore how the observation space in **NeuromechFly** can be used to simulate different sensory organs found in the fly. Specifically, we focus on two important sensory systems: **campaniform sensilla** and **vision**. Of course reproducing the exact response of those sensory modalities is far beyond the scope of your model and would requires hours of modelling work. Nevertheless, by using the observation data from NeuromechFly, we can get a first approximation of the response of these organs during the replayed behavior. 

In the following sections, we will explore how the observation space can be extracted and plotted and how it can relate to the response of the *Drosophila's* sensory organs:
- **Campaniform Sensilla**
- **Vision**

## 1. Campaniform Sensilla

### Overview
**Campaniform sensilla** are mechanoreceptors found on the legs, antennas, halteres and wings of flies that detect mechanical strain or stress on the cuticle. These sensors help the fly monitor forces acting on its legs during walking or flying. The neurons associated with campaniform sensilla respond when the cuticle is deformed due to external forces, providing proprioceptive feedback to control movement.

- **Anatomy**: These sensory organs are typically located in clusters on the legs, especially near the joints. Each sensillum consists of a dome-shaped cuticle that responds to strain, compressing and stimulating sensory neurons beneath it.
- **Sensory function**: Campaniform sensilla detect strain forces in the cuticule. In our model and in the biological fly, sources of cuticule strain are interactions with the environemnet and forces applied by the muscles. In **NeuromechFly** this strain could be approximated using a probably nonlinear combination of **contact forces** and **joint torques** from the observation space.

### Figure: Illustration of campaniform sensilla on a cockroach's tibia 

<figure>
    <img
    src="https://github.com/NeLy-EPFL/_media/blob/main/flygym/kinematic_replay/campanoform_sensilla.jpeg?raw=true"
    alt="rule_based"
    width="800"
    style="width: 510px; height: 337px; object-fit: cover; object-position: 0% 0;"
    />
    <figcaption>source: Tuthill, J. C., & Wilson, R. I. (2016). Mechanosensation and adaptive motor control in insects. Current Biology </figcaption>

</figure>


**References**:
- Tuthill, J. C., & Wilson, R. I. (2016). Mechanosensation and adaptive motor control in insects. Current Biology, 26(20), R1022-R1038.
- Zill, S., Schmitz, J., & Büschges, A. (2004). Load sensing and control of posture and locomotion. Arthropod structure & development, 33(3), 273-286.

---

### Ground reaction forces

In [ ]:
legs = [side + pos for pos in "FMH" for side in "LR"]

# For each leg get the indices of the tarsal segments
leg_tarsal_seg_contact_id = {
    leg: [i for i, tarsal_seg in enumerate(all_tarsi_links) if leg in tarsal_seg]
    for leg in legs
}
# For each get the norm of the sum of the contact forces on the tarsal segments
leg_contacts = np.linalg.norm(
    [
        np.sum(
            [
                [obs["contact_forces"][i] for obs in obs_list]
                for i in leg_tarsal_seg_contact_id[leg]
            ],
            axis=0,
        )
        for leg in legs
    ],
    axis=-1,
)
time = np.arange(len(leg_contacts[0])) * timestep

fig, axs = plt.subplots(len(legs), 1, figsize=(10, 10), tight_layout=True, sharey=True)
colors = plt.get_cmap("tab10", len(legs))
for i, leg in enumerate(legs):
    ax = axs[i]
    ax.plot(time, leg_contacts[i], label=leg, color=colors(i))
    ax.set_ylabel("Contact force [µN]")
    ax.legend(loc="upper right")
    ax.set_ylim(0, 10)

ax.set_xlabel("Time (s)")
fig.savefig(output_dir / "ground_contact_forces.png")

NOTE: In mujoco no units are imposed, consistency is the key. Lets walk through those. Units of distance are given in **mm** in our simulation, units of mass are given in **g** and units of time in **s**.

Hence the magnitude of the gravitational acceleration should be set to -98100 **$\frac{mm}{s^2}$**. 

Force are given in **$\frac{mm g}{s^2}$** equivalent to **$µN$**: micro Newtons.

Torques are given in **$µN mm$**.

All physical quantities can be derived in this way. 

### Joint torques

Here we will track the torque of all joints in a single leg.

In [ ]:
# Get indices corresponding to joints of the same leg
leg_joints_to_id = {
    leg: [i for i, joint in enumerate(monitored_joints) if leg in joint] for leg in legs
}
focus_leg = "LH"
# Extract torque values (index 2 corresponds to torque)
leg_torques = (
    np.array([obs["joints"][2, leg_joints_to_id[focus_leg]] for obs in obs_list]) * 1e9
)
# note the multiplication by 1e9 to convert to µN.mm (our implementation converted the torque to N.m but for consistency with the contact forces we convert it to µN.mm)

leg_joints = [joint for joint in monitored_joints if focus_leg in joint]

# Create subplots with shared x-axis, no overlapping y-labels, and tight layout
fig, axs = plt.subplots(
    len(leg_joints),
    1,
    figsize=(12, 10),
    sharex=True,
    gridspec_kw={"hspace": 0.5},
    tight_layout=True,
)

# Colors for the joints (tab10 colormap)
colors = plt.get_cmap("viridis", len(leg_joints))

# Loop through each joint and plot its torque data
for i, (ax, joint) in enumerate(zip(axs, leg_joints)):
    # Check if it's a passive or active joint
    label = joint.replace("joint_" + focus_leg, "")
    if not ("_" in label):
        label = label + "_pitch"
    if "Tarsus" in label and not "Tarsus1" in label:
        linestyle = "dashed"  # Passive joints (tarsal) use dotted lines
    else:
        linestyle = "solid"  # Active joints use solid lines

    # Plot the torque data with appropriate linestyle
    ax.plot(time, leg_torques[:, i], color=colors(i), linestyle=linestyle, label=label)

    # Remove the box around the plot (spines)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(True)
    ax.spines["bottom"].set_visible(True)
    ax.set_ylabel(joint.replace("joint_", "")[2:])

    # Disable y-tick labels to save space
    ax.tick_params(axis="y", which="both", left=True, right=False)

# Add a single y-axis label for torque (centered on the figure)
fig.text(0.04, 0.5, "Torque [µN.mm]", va="center", rotation="vertical", fontsize=10)

# Set the x-axis label on the last subplot
axs[-1].set_xlabel("Time (s)", fontsize=10)
axs[-1].tick_params(axis="x", labelsize=8)  # Smaller tick labels for x-axis

# Add a legend (outside the plot)
handles, labels = [], []
# add invisible black full and dashed lines to the legend to indicate active and passive joints
handles.append(plt.Line2D([0], [0], color="black", linestyle="solid", linewidth=1))
labels.append("Active joint")
handles.append(plt.Line2D([0], [0], color="black", linestyle="dashed", linewidth=1))
labels.append("Passive joint")
# add the actual joint labels
fig.legend(handles, labels, loc="upper right", bbox_to_anchor=(0.95, 0.95))

fig.suptitle(f"Torque values for {focus_leg} leg joints", fontsize=12)
fig.savefig(output_dir / "joint_torques.png")

## 2. Vision

A fly’s compound eye consists of ∼700–750 individual units called ommatidia arranged in a hexagonal pattern (see the left panel of the figure below from the [droso4schools project](https://droso4schools.wordpress.com/l4-enzymes/#5); see also [this article](https://azretina.sites.arizona.edu/node/789) from the Arizona Retina Project). To emulate this, we attached a color camera to each of our model’s compound eyes (top right panel). We then transformed each camera image into 721 bins, representing ommatidia. Based on previous studies, we assume a 270° combined azimuth for the fly’s field of view, with a ∼17° binocular overlap. Visual sensitivity has evolved to highlight ethologically relevant color spectra at different locations in the environment. Here, as an initial step toward enabling this heterogeneity in our model, we implemented yellow- and pale-type ommatidia—sensitive to the green and blue channels of images rendered by the physics simulator—randomly assigned at a 7:3 ratio (as reported in [Rister et al, 2013](https://pubmed.ncbi.nlm.nih.gov/23293281/)). Users can substitute the green and blue channel values with the desired light intensities sensed by yellow- and pale-type ommatidia to achieve more biorealistic chromatic vision.

<img src="https://github.com/NeLy-EPFL/_media/blob/main/flygym/vision_basics/vision.png?raw=true" alt="rule_based" width="800"/>

Neurmechfly v2 is largely oriented toward the modelling of higher order sensory modalities. With Neuromechfly v2 it is now possible to emulate the vision experienced by the fruitfly during behavior. This is again almost impossible to obtain without relying on kinematic relay.

Here is a representation of both the raw output of the cameras as well as the fly's simulated vision.

In [ ]:
from flygym.vision.visualize import visualize_visual_input

visualize_visual_input(
    fly.retina,
    output_dir / "retina_images.mp4",
    vision_list,
    raw_vision_list,
    np.ones(
        len(raw_vision_list), dtype=bool
    ),  # as we only saved the raw vision when rendered we set all to True
    playback_speed=play_speed,
)
plt.close("all")

# play the video
mp4 = open(output_dir / "retina_images.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(
    """
<video controls>
      <source src="%s" type="video/mp4">
</video>
"""
    % data_url
)